# Natural Language Processing

**THIS IS ONLY FOR 15-688 STUDENTS**

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based on a TFIDF matrix, and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essay written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution.  If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers . They are a particularly interesting data set, because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

In [2]:
import collections # optional, but we found the collections.Counter object useful
import itertools
import gzip
import re
import numpy as np
import scipy.sparse as sp
from testing.testing import test 

#Collaboration with: Sai Yogesh, Mehak Sikka, Daksha Bopanna

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18 .  Specifically, the "pg18.txt.gz" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [3]:
def load_federalist_corpus(filename):
    """ Load the federalist papers as a tokenized list of strings"""
    with gzip.open(filename, "rt", encoding="utf-8") as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in punctuation:
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers


You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `PAPERS` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some puncutation tokens, which have been modified to include spaces both before and after the punctuation. The `AUTHORS` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally the `NUMBERS` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

## Q1: Bag of words, and TFIDF

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

You should create the tfidf vector using numpy matrix operations and not use an existing implementation.

Important: make sure you do _not_ include the empty token `""` as one of your terms. 

In [4]:
TEST_DATA = ["the goal of this lecture is to explain the basics of free text processing",
             "the bag of words model is one such approach",
             "text processing via bag of words"]

def tfidf_test(tfidf_impl):
    # Simple starting test case:
    X_tfidf, words = tfidf_impl(["A A", "A", ""])
    test.equal(words, ["A"])
    test.equal(np.round(X_tfidf.todense().tolist(), 4).tolist(), [[0.8109], [0.4055], [0.]])
 
    # Little more complicated:
    X_tfidf, words = tfidf_impl(["A A A C", "A B", "C"])
    test.equal(sorted(words), list("ABC"))
    # Get word indices in order:
    X_tfidf = X_tfidf.todense()
    X_lookup = { w: X_tfidf[:,i].ravel() for i, w in enumerate(words)}

    # Check A, B, and C columns in order:
    test.equal(np.round(X_lookup["A"], 4).tolist(), [[1.2164, 0.4055, 0.    ]])
    test.equal(np.round(X_lookup["B"], 4).tolist(), [[0.    , 1.0986, 0.    ]])
    test.equal(np.round(X_lookup["C"], 4).tolist(), [[0.4055, 0.    , 0.4055]])
 
    # With test data from above
    X_tfidf, words = tfidf_impl(TEST_DATA)
    test.equal(X_tfidf.shape, (3, 19))
    test.equal(set(words), {'one', 'bag', 'goal', 'explain', 'approach', 'to', 'processing', 'this', 'the', 'model', 'basics', 'free', 'words', 'such', 'is', 'text', 'lecture', 'via', 'of'})
    test.equal(X_tfidf[0, words.index('explain')], 1.0986122886681098)

@test
def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """

    document_words=[doc.split() for doc in docs]
    req_vocab=list(set(sum(document_words,[])))
    vocab_dict={k:i for i,k in enumerate(req_vocab)}
    r_ind=[k for k in range(len(docs))]
    
    new_dict={}
    word_dict={}
    for i,dw in enumerate(document_words):
        word_dict[i]=dict(collections.Counter(document_words[i]))
        new_dict[i]=dict(collections.Counter(set(document_words[i])))
    u_word_dict=[]
    #print(u_word)
    for key,value in new_dict.items():
            u_word_dict.extend(value)
    u_word_dict=dict(collections.Counter(u_word_dict))
    row_indices=np.array([r_ind[key] for key,value in word_dict.items() for key1,value1 in value.items()])
    column_indices=np.array([vocab_dict[key3] for key2,value2 in word_dict.items() for key3,value3 in value2.items()])
    values=np.array([value4*np.log(len(docs)*1.0/u_word_dict[key4]) for key3,value3 in word_dict.items() for key4,value4 in value3.items()])

    matrix = sp.coo_matrix((values, (row_indices, column_indices)), shape=(len(docs),len(req_vocab)))
    matrix = matrix.tocsr()
    matrix.eliminate_zeros()
    
    
    return (matrix,req_vocab)

### TESTING tfidf: PASSED 9/9
###



Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>

Next, implement the following simple function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [5]:
def cosine_similarity_test(cosine_similarity_impl):
     # Little more complicated:
    X_tfidf, words = tfidf(["A A A C", "A B", "C"])
    M = cosine_similarity_impl(X_tfidf)
    if M is None:
        return test.true(False)
    test.true(np.allclose(M, np.matrix([[1., 0.32847358, 0.31622777], [0.32847358, 1., 0.], [0.31622777, 0., 1.]])))

    # Test data
    X_tfidf, words = tfidf(TEST_DATA)
    M = cosine_similarity_impl(X_tfidf)
    test.true(np.allclose(M, 
        np.matrix([[1., 0.06796739, 0.07771876], [0.06796739, 1., 0.10281225], [0.07771876, 0.10281225, 1.]])))

@test
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """
    X=X.todense()
    X_cosine=X/np.linalg.norm(X,axis=1)[:,None]
    M=np.array(X_cosine@X_cosine.T)
    return M

    pass

### TESTING cosine_similarity: PASSED 2/2
###



Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Fill out the following function to compute and return these averages.

Hints:

1. fit a single TFIDF encoding to all papers and transform all papers using it before computing the similarity measure
2. for the cosine similarity to be useful when comparing documents, they must all be encoded the same way. Transform all papers together before comparing cosine similarity.
3. the unknown papers have author=`("HAMILTON","MADISON")`;

In [6]:
def author_cosine_similarity_test(author_cosine_similarity_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    hamilton_mcs, madison_mcs, jay_mcs = author_cosine_similarity_impl(papers, authors)
    test.equal(np.round(jay_mcs, 4), 0.0649)

@test
def author_cosine_similarity(docs, authors):
    """
    Return a tuple of average cosine similarities between all the known papers for a given author
    and all the unknown papers.
    
    Args:
        docs: list of strings, where each string represents a space-separated
              document
        authors: list of lists, which each list contains the author (or potential authors) of a given document
    
    Returns: tuple: (hamilton_mcs, madison_mcs, jay_mcs)
        hamilton_mcs: Average cosine similarity between all the known Hamilton papers and all the unknown papers.
        madison_mcs: Average cosine similarity between all the known Madison papers and all the unknown papers.
        jay_mcs: Average cosine similarity between all the known Jay papers and all the unknown papers.
    """
    t = tfidf(docs)[0]
#     print(t)
    M = cosine_similarity(t)
#     print(M)
    unknown = []
    hamilton = []
    madison =[]
    jay = []
#     print(authors)
    for i in range(len(authors)):
#         print(i)
        if (len(authors[i]))>1:
            unknown.append(i)
#         print(authors[i])
        if authors[i] == ('HAMILTON',):
            hamilton.append(i)
        if authors[i] == ('MADISON',):
            madison.append(i)
        if authors[i] == ('JAY',):
            jay.append(i)
    
#     print(hamilton)
    hamilton_mcs_list = []
    madison_mcs_list = []
    jay_mcs_list = []
    
    for j in unknown:
        for i in hamilton:
#             print(M.item((j,i)))
            hamilton_mcs_list.append(M.item((j,i)))
    hamilton_mcs=sum(hamilton_mcs_list)/len(hamilton_mcs_list)
    
    for j in unknown:
        for i in madison:
            madison_mcs_list.append(M.item((j,i)))
    madison_mcs=sum(madison_mcs_list)/len(madison_mcs_list)
    
    for j in unknown:
        for i in jay:
            jay_mcs_list.append(M.item((j,i)))
    jay_mcs=sum(jay_mcs_list)/len(jay_mcs_list)
    
    return (hamilton_mcs, madison_mcs, jay_mcs)
    
    pass

### TESTING author_cosine_similarity: PASSED 1/1
###



## Q2: N-gram language models

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class:

In [9]:
def LanguageModel_test(LanguageModel_impl):
#     papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
#     l_hamilton = LanguageModel_impl(papers, 3)
#     test.equal(l_hamilton.counts["privilege of"], {'being': 1, 'proposing': 1, 'residence': 1, 'originating': 1, 'paying': 1, 'the': 1})
#     test.equal(l_hamilton.count_sums["privilege of"], 6)
#     test.equal(np.round(l_hamilton.perplexity(papers[0]), 4), 15.1144)
#     # Simple Test
    l_mini = LanguageModel_impl(["a a a a a", "a b a b"], 2)
#     # Counts
#     test.equal(l_mini.counts, {'a': {'a': 4, 'b': 2}, 'b': {'a': 1}})
#     test.equal(l_mini.count_sums, {'a': 6, 'b': 1})
    # Simple perplexity checks:
#     test.equal(round(l_mini.perplexity("a a"), 3), 1.500)
#     test.equal(round(l_mini.perplexity("a a a"), 3), 1.500)
#     test.equal(round(l_mini.perplexity("a b"), 3), 3.000)
#     test.equal(round(l_mini.perplexity("a a b"), 3), 2.121)
#     test.equal(round(l_mini.perplexity("b a"), 3), 1.001)
#     # Check perplexity when the predecessor is present but not the successor
    test.equal(round(l_mini.perplexity("a c"), 3), 6003.0)
#     # Check perplexity when the predecessor is not present
#     test.equal(round(l_mini.perplexity("c a"), 3), 3.000)



@test
class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.
        
        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """
        
        self.counts = collections.defaultdict(dict) # Dict from space-separated "previous words" to a Dict of (next word, count). 
        self.count_sums = {} # Dict from space-separated "previous words" to the total number of times they appear
        
        count_sums={}
        count={}
        for doc in docs:
            doc=doc.split()
            for i in range(len(doc)-n+1):
                key1=' '.join(doc[i:i+n-1])
                key2=''.join(doc[i+n-1:i+n])
                if key1 in count:
                    count_sums[key1]+=1
                    if key2 in count[key1]:
                        count[key1][key2]+=1 
                    else:
                        count[key1][key2]=1
                else:
                    count[key1]={}
                    count[key1][key2]=1
                    count_sums[key1]=1
                    
        self.count_sums=count_sums
        self.counts=count
#         print(self.counts)
        self.dictionary=set(' '.join(docs).split())
        self.n=n
        
        pass
    
    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.
        
        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing
            
        Note: for the purposes of smoothing, the dictionary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.
            
        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluted
                        under the model.
        """
        
        text=text.split()
        D=len(set(text).union(self.dictionary))
        sum_prob=0
        
        for i in range(len(text)-self.n+1):
            num=alpha
            denom=alpha*D
            key1=' '.join(text[i:i+self.n-1])    
            key2=''.join(text[i+self.n-1:i+self.n])
            if key1 in self.counts:
                if key2 in self.counts[key1]:
                    num+=self.counts[key1][key2]
#                     print("num", num)
                denom+=self.count_sums[key1]
#                 print(denom)
            sum_prob=sum_prob+np.log(num/denom)
#             print(sum_prob)
            
        perplexity=np.exp(-sum_prob/(len(text)-(self.n-1)))
        
        return perplexity
        
        pass

### TESTING LanguageModel: PASSED 1/1
###



### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You can store the information any way you want, but we did this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later function, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen. We also build a `self.dictionary` variable, which is just a `set` object containing all the unique words in across the entire set of input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  Use the formula for perplexity from the class nodes (which is actually not exact, since it only so, being careful to not multiply togther probabilites that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

You'll want to be careful about dictionary sizes when it comes to the Laplace smoothing term: make sure your dictionary size $D$ is equal to the total number of unique words that occur in either the original data used to build the language model _or_ in the text we are evaluating the perplexity of (so the size of the union of the two).

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`). Fill in the following function to calculate and return the mean perplexities.

In [91]:
def mean_perplexity_test(mean_perplexity_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    perp_hamilton, perp_madison, perp_jay = mean_perplexity_impl(papers, authors)
    test.equal(np.round(perp_hamilton, 4), 1941.3852)

@test
def mean_perplexity(docs, authors):
    """
    Evaluate the mean of the perplexity of the unknown Federalist papers for the language models
    from each of the three authors (again, using n=3 and alpha=1e-3)

    Args:
        docs: list of strings, where each string represents a space-separated document
        authors: list of lists, which each list contains the author (or potential authors) of a given document

    Returns: tuple: (perp_hamilton, perp_madison, perp_jay)
        perp_hamilton: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Hamilton
        perp_madison: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Madison
        perp_jay: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Jay
    """
    unknown = []
    hamilton = []
    madison =[]
    jay = []
    
    #Get all docs that are unknown
    #Get all docs for Madison, Hamilton and Jay
    for i in range(len(authors)):
        if (len(authors[i]))>1:
            unknown.append(docs[i])
        elif authors[i] == ('HAMILTON',):
            hamilton.append(docs[i])
        elif authors[i] == ('MADISON',):
            madison.append(docs[i])
        elif authors[i] == ('JAY',):
            jay.append(docs[i])
            
    #Use each of those docs (and only the authors docs) to create three language models: H, M, and J
    H_lm = LanguageModel(hamilton, 3)
    M_lm = LanguageModel(madison, 3)
    J_lm = LanguageModel(jay, 3)
    
    #Iterate through all unknown docs and record the perplexity score for each of these docs against each of the models
    #Average each models' perplexity scores (i.e get average of ( [ list of hamiltons scores]) for hamilton)
    perp=0.0
    for i in range(len(unknown)):
        perp+= H_lm.perplexity(unknown[i], alpha=1e-3)
    perp_hamilton = perp/len(unknown)      
    
    perp=0.0
    for i in range(len(unknown)):
        perp+= M_lm.perplexity(unknown[i], alpha=1e-3)
    perp_madison = perp/len(unknown)
        
    perp=0.0
    for i in range(len(unknown)):
        perp+= J_lm.perplexity(unknown[i], alpha=1e-3)
    perp_jay = perp/len(unknown)
    
    return (perp_hamilton, perp_madison, perp_jay)
    
    pass

### TESTING mean_perplexity: PASSED 1/1
###

